<a href="https://colab.research.google.com/github/dmitryglhf/jupyter-projects/blob/main/lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Lib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score as roc_auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve

import torch
import torch.nn as nn
import copy
import torch.optim as optim
import tqdm

### Householder, QR, Eig, SVD


In [ ]:
def householder(a):
    v = a.copy()
    v[0] += np.sign(v[0]) * np.linalg.norm(a)
    v_dot = np.dot(v, v)

    if v_dot == 0:
        H = np.eye(len(a))  # если v_dot равно нулю, то H будет единичной матрицей
    else:
        H = np.eye(len(a)) - 2 * np.outer(v, v) / v_dot  # вычисление матрицы Хаусхолдера
    return H


def get_qr(A):
    n = A.shape[0]
    Q = np.eye(n)
    R = A.copy()

    for j in range(n):
        a = R[j:, j]  # getting vector
        H = householder(a)  # getting Householder matrix

        R[j:, j:] = H @ R[j:, j:]  # change R
        Q[:, j:] = Q[:, j:] @ H.T  # change Q

    return Q, R


def eig(A, e=1e-10, max_iter=1000):
    Ak = A.copy()
    eig_vectors = np.eye(A.shape[0])

    for i in range(max_iter):
        Q, R = get_qr(Ak)
        Ak = np.dot(R, Q)
        eig_vectors = np.dot(eig_vectors, Q)

        off_diag_norm = np.sqrt(np.sum(np.tril(Ak, -1) ** 2))
        if off_diag_norm < e:
            break

    eig_values = np.diag(Ak)

    return eig_values, eig_vectors


def svd(A):
    # Вычисляем U и собственные значения для A A^T
    temp_U = A @ A.T
    eig_values_U, U = eig(temp_U)

    # Сортируем собственные значения и векторы по убыванию
    idx_U = np.argsort(eig_values_U)[::-1]
    eig_values_U = eig_values_U[idx_U]
    U = U[:, idx_U]

    # Матрица Sigma
    sigma = np.zeros((A.shape[0], A.shape[1]))
    for i in range(min(A.shape[0], A.shape[1])):
        sigma[i, i] = np.sqrt(eig_values_U[i]) if eig_values_U[i] > 0 else 0

    # Вычисляем V^T
    sigma_inv = np.zeros_like(sigma.T)
    for i in range(min(A.shape[0], A.shape[1])):
        if sigma[i, i] > 0:
            sigma_inv[i, i] = 1 / sigma[i, i]

    V_T = sigma_inv @ U.T @ A

    # Результат
    res = U @ sigma @ V_T

    print(f'NASH KOD Origin matrix A: \n{A} \n ')
    print(f'Матрица левых сингулярных векторов U: \n {U} \n ')
    print(f'Матрица сингулярных чисел Sigma: \n {sigma} \n ')
    print(f'Матрица правых сингулярных векторов V_T: \n {V_T} \n ')
    print(f'Исходная матрица: \n {res} \n ')

    return U, sigma, V_T



### PCA

In [ ]:
def pca(X):
  U, S, V_T = svd(X)

  # finding numer of PC
  sum = S[0][0]
  explained_ratio = sum / np.sum(np.diag(X))
  i = 1
  while(explaned_ratio <= 0.95):
    sum += S[i][i]
    explained_ratio = sum / np.sum(np.diag(X))
    i += 1

  # projection
  V_T_k = V_T[:i, :]  # берем первые k строк матрицы V_T
  X_pca = X @ V_T_k.T

  return X_pca

### Test - funct

In [ ]:
def svd_test(A):
    U, S, V = svd(A)

    #U, S, V = np.linalg.svd(A)
    S = np.diag(S)

    #res = np.dot(U, np.dot(S, V))
    #res = np.dot(np.dot(U, S), V)

    #V[2][0] = -0.43042935
    #V[2][1] = 0.79483601
    #V[2][2] = -0.42774559

    #res = U @ S @ V
    print(f'NUMPY Origin matrix A: \n{A} \n ')
    print(f'Матрица левых сингулярных векторов U: \n {U} \n ')
    print(f'Матрица сингулярных чисел Sigma: \n {S} \n ')
    print(f'Матрица правых сингулярных векторов V_T: \n {V} \n ')
    #print(f'Исходная матрица: \n {res} \n ')


def qr_test(X):
    test_X = X @ X.T
    Q, R = get_qr(test_X)

    print(f'NUMPY Origin matrix A = X @ X.T: \n{test_X} \n ')
    print(f'Матрица Q: \n {Q} \n ')
    print(f'Матрица R: \n {R} \n ')
    print(f'Исходная матрица A = Q @ R: \n {Q @ R} \n ')

### Test

In [ ]:
# test 1
#A = np.array([[1, 2, 3], [5, 7, 8], [3, 6, 8]], dtype=float) # все ок

# test 2
#A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=float) # нижняя строка - нули ???

# test 3
#A = np.array([[7, 2, -5], [-9, 8, -5], [24, -6, 8]], dtype=float) # все ок

# test 4 - прямоугольная матрица - нижняя строка - нули ???

A = np.array([[3, 5], [9, 4], [4, 5], [8, 9]], dtype = float)
'''
A = np.array([[1, 0, 0, 0, 2],
              [0, 0, 3, 0, 0],
              [0, 0, 0, 0, 0],
              [0, 4, 0, 0, 0]], dtype=float)
'''

#qr_test(A)
svd_test(A)
svd(A)

NUMPY Origin matrix A: 
[[3. 5.]
 [9. 4.]
 [4. 5.]
 [8. 9.]] 
 
Матрица левых сингулярных векторов U: 
 [[ 0.32200951 -0.42290278  0.56864182 -0.62778149]
 [ 0.53673164  0.81588836  0.08454905 -0.19772905]
 [ 0.36430103 -0.24802239 -0.81236006 -0.38189107]
 [ 0.68957507 -0.30653598  0.09782167  0.64880878]] 
 
Матрица сингулярных чисел Sigma: 
 [17.37704426  3.87792891] 
 
Матрица правых сингулярных векторов V_T: 
 [[ 0.73490162  0.67817373]
 [ 0.67817373 -0.73490162]] 
 


(array([[ 0.32200951, -0.42290278,  0.56864182, -0.62778149],
        [ 0.53673164,  0.81588836,  0.08454905, -0.19772905],
        [ 0.36430103, -0.24802239, -0.81236006, -0.38189107],
        [ 0.68957507, -0.30653598,  0.09782167,  0.64880878]]),
 array([[17.37704426,  0.        ],
        [ 0.        ,  3.87792891],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]]),
 array([[ 0.73490162,  0.67817373],
        [ 0.67817373, -0.73490162]]))

### NN


#### Data

In [ ]:
data = pd.read_csv('cancer_tumor_data_features.csv')
columns_to_drop = ['id', 'Unnamed: 32']
data.drop(columns_to_drop, axis=1, inplace=True)

X = data.drop('diagnosis', axis=1)
y = data.diagnosis

In [ ]:
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)
print(encoder.classes_)

#### Build model

In [ ]:
X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

In [ ]:
class MyNN(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.hidden = nn.Linear(n, n)
        self.relu = nn.ReLU()
        self.output = nn.Linear(n, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.hidden(x))
        x = self.sigmoid(self.output(x))
        return x

In [ ]:
model = MyNN(30)
print(sum([x.reshape(-1).shape[0] for x in model.parameters()]))

In [ ]:
def model_train(model, X_train, y_train, X_val, y_val):
    # loss function and optimizer
    loss_fn = nn.BCELoss()  # binary cross entropy
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    n_epochs = 250   # number of epochs to run
    batch_size = 10  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)

    # Hold the best model
    best_acc = - np.inf   # init to negative infinity
    best_weights = None

    for epoch in range(n_epochs):
        model.train()
        with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size]
                y_batch = y_train[start:start+batch_size]
                # forward pass
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                acc = (y_pred.round() == y_batch).float().mean()
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )
        # evaluate accuracy at end of each epoch
        model.eval()
        y_pred = model(X_val)
        acc = (y_pred.round() == y_val).float().mean()
        acc = float(acc)
        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())
    # restore model and return best accuracy
    model.load_state_dict(best_weights)
    return best_acc

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)

In [ ]:
%%time
acc = model_train(model, X_train, y_train, X_test, y_test)
print(f"Final model accuracy: {acc*100:.2f}%")